# Get the permissions 

This Notebook will get the current Server Role Membership for all Logins and the database role membership for all user database users and the securables for all databases. It uses T-SQL

## Server Roles Membership for all logins

In [1]:
WITH CTE_Role (name,role,type_desc)
AS
(SELECT PRN.name,
srvrole.name AS [role] , 
Prn.Type_Desc 
FROM sys.server_role_members membership 
INNER JOIN (SELECT * FROM sys.server_principals  WHERE type_desc='SERVER_ROLE') srvrole 
ON srvrole.Principal_id= membership.Role_principal_id 
RIGHT JOIN sys.server_principals  PRN 
ON PRN.Principal_id= membership.member_principal_id WHERE Prn.Type_Desc NOT IN ('SERVER_ROLE') AND PRN.is_disabled =0

UNION ALL

SELECT  p.[name], 'ControlServer' ,p.type_desc AS loginType FROM sys.server_principals p
  JOIN sys.server_permissions Sp
   ON p.principal_id = sp.grantee_principal_id WHERE sp.class = 100
  AND sp.[type] = 'CL'
  AND state = 'G' )
SELECT 
@@SERVERNAME AS ServerName,
name,
Type_Desc ,
CASE WHEN [public]=1 THEN 'Y' ELSE 'N' END AS 'Public',
CASE WHEN [sysadmin] =1 THEN 'Y' ELSE 'N' END AS 'SysAdmin' ,
CASE WHEN [securityadmin] =1 THEN 'Y' ELSE 'N' END AS 'SecurityAdmin',
CASE WHEN [serveradmin] =1 THEN 'Y' ELSE 'N' END AS 'ServerAdmin',
CASE WHEN [setupadmin] =1 THEN 'Y' ELSE 'N' END AS 'SetupAdmin',
CASE WHEN [processadmin] =1 THEN 'Y' ELSE 'N' END AS 'ProcessAdmin',
CASE WHEN [diskadmin] =1 THEN 'Y' ELSE 'N' END AS 'DiskAdmin',
CASE WHEN [dbcreator] =1 THEN 'Y' ELSE 'N' END AS 'DBCreator',
CASE WHEN [bulkadmin] =1 THEN 'Y' ELSE 'N' END AS 'BulkAdmin' ,
CASE WHEN [ControlServer] =1 THEN 'Y' ELSE 'N' END AS 'ControlServer' 
FROM CTE_Role
PIVOT(
COUNT(role) FOR role IN ([public],[sysadmin],[securityadmin],[serveradmin],[setupadmin],[processadmin],[diskadmin],[dbcreator],[bulkadmin],[ControlServer])
) AS pvt WHERE Type_Desc NOT IN ('SERVER_ROLE')ORDER BY name,type_desc

(18 rows affected)

Total execution time: 00:00:00.148

ServerName,name,Type_Desc,Public,SysAdmin,SecurityAdmin,ServerAdmin,SetupAdmin,ProcessAdmin,DiskAdmin,DBCreator,BulkAdmin,ControlServer
SQL2017N5,##MS_AgentSigningCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,##MS_PolicySigningCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,Y
SQL2017N5,##MS_SmoExtendedSigningCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,##MS_SQLAuthenticatorCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,##MS_SQLReplicationSigningCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,##MS_SQLResourceSigningCertificate##,CERTIFICATE_MAPPED_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,NT AUTHORITY\SYSTEM,WINDOWS_LOGIN,N,N,N,N,N,N,N,N,N,N
SQL2017N5,NT Service\MSSQLSERVER,WINDOWS_LOGIN,N,Y,N,N,N,N,N,N,N,N
SQL2017N5,NT SERVICE\SQLSERVERAGENT,WINDOWS_LOGIN,N,Y,N,N,N,N,N,N,N,N
SQL2017N5,NT SERVICE\SQLTELEMETRY,WINDOWS_LOGIN,N,N,N,N,N,N,N,N,N,N


## Database Role Members for all users on all user databases

In [2]:
SET NOCOUNT ON

CREATE TABLE #DatabaseRoleMemberShip
   (
        Username VARCHAR(100),
        Rolename VARCHAR(100),
        Databasename VARCHAR(100)
        
    )DECLARE @Cmd AS VARCHAR(MAX)DECLARE @PivotColumnHeaders VARCHAR(4000)           SET @Cmd = 'USE [?] ;insert into #DatabaseRoleMemberShip
select u.name,r.name,''?'' from sys.database_role_members RM inner join
sys.database_principals U on U.principal_id=RM.member_principal_id
inner join sys.database_principals R on R.principal_id=RM.role_principal_id
where u.type<>''R'''EXEC sp_MSforeachdb @command1=@cmd

SELECT  @PivotColumnHeaders =                        
  COALESCE(@PivotColumnHeaders + ',[' + CAST(rolename AS VARCHAR(MAX)) + ']','[' + CAST(rolename AS VARCHAR(MAX))+ ']'                     
  )                     
  FROM (SELECT DISTINCT rolename FROM #DatabaseRoleMemberShip )a ORDER BY rolename  ASC


SET @Cmd = '
select
@@SERVERNAME AS ServerName,
DatabaseName,UserName,'+@PivotColumnHeaders+'
from
(
select   * from #DatabaseRoleMemberShip) as p
pivot
(
count(rolename  )
for rolename in ('+@PivotColumnHeaders+') )as pvt'

EXECUTE(@Cmd )        

DROP TABLE #DatabaseRoleMemberShip 

Commands completed successfully.

Total execution time: 00:00:00.313

ServerName,DatabaseName,UserName,db_owner,db_ssisoperator,dc_admin,PolicyAdministratorRole,SQLAgentUserRole,ssis_admin
SQL2017N5,ADonutFactory,dbo,1,0,0,0,0,0
SQL2017N5,ADonutFactory,THEBEARD\akamman,1,0,0,0,0,0
SQL2017N5,ADonutFactory,THEBEARD\gsartori,1,0,0,0,0,0
SQL2017N5,ADonutFactory,THEBEARD\wdurkin,1,0,0,0,0,0
SQL2017N5,ASausageFactory,dbo,1,0,0,0,0,0
SQL2017N5,ASausageFactory,THEBEARD\akamman,1,0,0,0,0,0
SQL2017N5,ASausageFactory,THEBEARD\gsartori,1,0,0,0,0,0
SQL2017N5,ASausageFactory,THEBEARD\wdurkin,1,0,0,0,0,0
SQL2017N5,ASockFactory,dbo,1,0,0,0,0,0
SQL2017N5,ASockFactory,THEBEARD\akamman,1,0,0,0,0,0


## All User Database Role Securables

In [3]:
EXECUTE master.sys.sp_MSforeachdb '
IF ''?'' <> ''master'' AND ''?'' <> ''model'' AND ''?'' <> ''msdb'' AND ''?'' <> ''tempdb''
		BEGIN
USE [?]; 

SELECT DISTINCT 
                @@SERVERNAME AS ServerName,
                DB_NAME() AS DatabaseName,
                rp.name AS RoleName, 
                ObjectType = rp.type_desc, 
                PermissionType = pm.class_desc, 
                pm.permission_name, 
                pm.state_desc, 
                ObjectType = CASE 
                               WHEN obj.type_desc IS NULL 
                                     OR obj.type_desc = ''SYSTEM_TABLE'' THEN 
                               pm.class_desc 
                               ELSE obj.type_desc 
                             END, 
                s.Name as SchemaName,
                [ObjectName] = Isnull(ss.name, Object_name(pm.major_id)) 
FROM   sys.database_principals rp 
       INNER JOIN sys.database_permissions pm 
               ON pm.grantee_principal_id = rp.principal_id 
       LEFT JOIN sys.schemas ss 
              ON pm.major_id = ss.schema_id 
       LEFT JOIN sys.objects obj 
              ON pm.[major_id] = obj.[object_id] 
       LEFT JOIN sys.schemas s
              ON s.schema_id = obj.schema_id
WHERE  rp.type_desc = ''DATABASE_ROLE'' 
       AND pm.class_desc <> ''DATABASE'' 
ORDER  BY rp.name, 
          rp.type_desc, pm.class_desc 
          END
          '

Commands completed successfully.

Total execution time: 00:00:00.289

ServerName,DatabaseName,RoleName,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_columns
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_objects
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_parameters
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_sql_modules
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_views
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,allocation_units
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assemblies
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_files
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_modules
SQL2017N5,SqlWatch,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_references


ServerName,DatabaseName,RoleName,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,CLR_STORED_PROCEDURE,internal,get_server_account
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,add_data_tap
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,add_data_tap_by_guid
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,clear_object_parameter_value
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,create_environment
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,create_environment_reference
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,create_environment_variable
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,create_execution
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,delete_environment
SQL2017N5,SSISDB,public,DATABASE_ROLE,OBJECT_OR_COLUMN,EXECUTE,GRANT,SQL_STORED_PROCEDURE,catalog,delete_environment_reference


ServerName,DatabaseName,RoleName,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_columns
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_objects
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_parameters
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_sql_modules
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_views
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,allocation_units
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assemblies
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_files
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_modules
SQL2017N5,ASockFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_references


ServerName,DatabaseName,RoleName,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_columns
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_objects
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_parameters
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_sql_modules
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_views
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,allocation_units
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assemblies
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_files
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_modules
SQL2017N5,ASausageFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_references


ServerName,DatabaseName,RoleName,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_columns
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_objects
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_parameters
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_sql_modules
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_views
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,allocation_units
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assemblies
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_files
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_modules
SQL2017N5,ADonutFactory,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_references
